# **0. PREPARACIÓN DE LOS DATOS Y SPLIT DEL DATASET**
Para este proyecto de deteccion de spoofing en sistemas ADB-S voy a estar usando el siguiente dataset de *Mendeley Data* : *ADS-B Message Injection Attacks Dataset*

## IMPORTAR Y CARGAR

In [521]:
import pandas as pd 
from sklearn.model_selection import train_test_split
#subir el dataset
df = pd.read_csv("Dataset ADB-S.csv")
#ver las medidas del dataset y su estructura (previsualizándolo)
print(f"Dataset con {df.shape[0]} filas y {df.shape[1]} columnas")
display(df.head())

Dataset con 22316 filas y 18 columnas


,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,rss,doppler,label
0,1627918130,0c20b6,41.073349,-73.930298,194.919295,231.859760,0.0,CMP312,False,False,1364.0,10363.2,10675.62,1627918130,1627918130,-85.607674,588.045536,0
1,1627918140,0c20b6,41.062500,-73.948425,194.833737,231.647341,0.0,CMP312,False,False,1364.0,10363.2,10675.62,1627918140,1627918140,-85.601463,827.517243,0
2,1627918150,0c20b6,41.051393,-73.966952,195.153367,231.528897,0.0,CMP312,False,False,1364.0,10363.2,10675.62,1627918150,1627918150,-85.609035,774.034419,0
3,1627918160,0c20b6,41.040918,-73.984368,195.153367,231.528897,0.0,CMP312,False,False,1364.0,10363.2,10675.62,1627918160,1627918160,-85.631899,790.374778,0
4,1627918170,0c20b6,41.035418,-74.006463,195.237429,251.741020,0.0,CMP312,False,False,1364.0,10363.2,10675.62,1627918169,1627918170,-85.744771,-94.629770,1


## SPLIT
Haré la siguiente división del dataset:

- 60% Train
- 20% Val (validation)
- 20% Test

De esta froma me aseguro de que tengo una parte del dataset para entrenar el modelo (trai), una específica para ajustar hiperparámetros (val) y por último una para probar el modelo y ver su rendimiento y la exactitud de sus predicciones (test).

La variable *icao24* representa la "matrícula" O "ID" de cada aeronave. Para cada aeronave tenemso multiples registros a lo largo del tiempo. Hacer el split por esta variable nos asegura que cada aeronave está solamente en un dataframe (train, test o val) y que por lo tanto el modelo no está aprendiendo los 'id' de la aeronave y clasificando por ellos, sino que está aprendiendo y clasificando sesgún el comportamiento. 


In [522]:
aeronaves = df['icao24'].unique()
#Divido de la siguiente forma: 60% train_ids, 40% rest_ids (20% val_ids and 20% test_ids)
train_ids, rest_ids = train_test_split(aeronaves, test_size=0.4, random_state=42)
val_ids, test_ids = train_test_split(rest_ids, test_size=0.5, random_state=42)
#creo los nuevos dataframes
df_train = df[df['icao24'].isin(train_ids)].copy()
df_val = df[df['icao24'].isin(val_ids)].copy()
df_test = df[df['icao24'].isin(test_ids)].copy()

print(f"Número de filas en cada dataframe: Train ({len(df_train)}), Val ({len(df_val)}), Test ({len(df_test)})")

Número de filas en cada dataframe: Train (13778), Val (4334), Test (4204)


## Ordenar el dataframe de train por órden cronológico

Solamente ordeno este dataframe (y no test y val) ya que es con el que estoy trabajando. 
Al ordenar los datos de cada aeronave por órden cronológico podemos tener la trayectoria paso a paso, lo que nos permite realizar un mejor estudio de los datos.

In [523]:
df_train = df_train.sort_values(by=['icao24', 'time']).reset_index(drop=True)

# **1. ESTUDIAR LAS VARIABLES** 

Importo paquetes y librerías

In [524]:
import matplotlib.pyplot as plt
import seaborn as sns

# Definición de clases

- **Label 0: Normal** : aeronave real y legítima
- **Label 1: modificación de la trayectoria** : ataque sobre una aeronave real en el que se modifican las coordenadas de la trayectoria (lat/lon)
- **Label 2: aeronave fantasma** : aeronave que no existe con patrones de vuelo artificiales
- **Label 3: deriva de velocidad** : la velocidad reportada no es la real, y por tanto no coincide con el movimiento de la aeronave

In [525]:
print("\nDistribución de las clases:")
print("\nNúmero de casos de cada clase (label)")
print(df_train['label'].value_counts())
print("\nPorcentaje de cada clase")
print(df_train['label'].value_counts(normalize=True) * 100)


Distribución de las clases:

Número de casos de cada clase (label)
label
0    6722
2    3977
1    1544
3    1535
Name: count, dtype: int64

Porcentaje de cada clase
label
0    48.787923
2    28.864857
1    11.206271
3    11.140949
Name: proportion, dtype: float64


# Duración media de los trayectos por clase

Calcular para cada aeronave y label la duración del trayecto en segundos (hay que tener en cuenta que es el trayecto que se detecta desde cada receptor, no es trayecto total de la aeronave). Para cada clase se calcula la duración media.

In [526]:
df = df_train.sort_values(['icao24', 'time'])

df['new_flight'] = (
    (df['onground'].shift(1) == True) & (df['onground'] == False)
) | (
    df['time'].diff() > 600  #10 min pq los mensajes se envía cada 0.5 / 1 seg
) | (
    df['icao24'] != df['icao24'].shift(1)
)

df['flight_id'] = df.groupby('icao24')['new_flight'].cumsum()

duraciones = df.groupby(['icao24', 'flight_id', 'label'])['time'].agg(lambda x: x.max() - x.min())
avg_duracion = duraciones.groupby('label').mean()
print(avg_duracion)


label
0     453.902439
1     371.428571
2    2658.125000
3     322.666667
Name: time, dtype: float64


# Análisis de la duración de vuelo

La duración calculada representa el ToT (Time-on-Target), que es el tiempo total que la aeronave está dentro del area de covertura del receptor (el rango donde la antena puede detectar señales).

- **Label 0**: Representa el tiempo estánder que una aeronave real tarda en cruzar el área de cobertura del receptor
- **Label 1**: Bastante realista y normal, ya que este ataque solamente modifica las coordenadas de una aeronave real, por lo que la duración del trayecto está subordinada a la presencia física de la aeronave víctima
- **Label 2**: El tiempo medio es *muy superior* al del resto de casos, lo que indica claramente al intención del atacante de maximizar los daños. Estas aeronaves fantasmas están programadas para hacer un bucle o mantenerse flotando dentro del alcance del radar y así saturar el sistema
- **Label 3**: Dado que el ataque simplemente modifica señales en vivo acerca de la velocidad, no hay apenas cambios y se asemeja mucho a las métricas para un vuelo legítimo

## Valores nulos

In [527]:
def nulos_f(df):
    nulos = df.isnull().sum()
    porcent = (nulos / len(df)) * 100
    return pd.DataFrame({'nulos': nulos, 'porcentaje': porcent})
nulos_f(df_train)

,nulos,porcentaje
time,0,0.000000
icao24,0,0.000000
lat,435,3.157207
lon,435,3.157207
velocity,1891,13.724779
heading,1891,13.724779
vertrate,1891,13.724779
callsign,139,1.008855
onground,0,0.000000
spi,0,0.000000


## Detección de variables que no aportan información

Miro los porcentajes de los valores más frecuentes para cada columna, si el valor dominante aparece más del 99% de los casos, se puede asegurar que es irrelevante y no aporta información, y por tanto se eliminará esa variable.

In [ ]:
no_info_var = (df_train.apply(lambda col: col.value_counts(normalize=True).iloc[0]).sort_values(ascending=False))

print("Porcentaje de aparición del valor dominante en cada variable")
print(no_info_var)

eliminar = no_info_var[no_info_var > 0.99].index.tolist()

print("\nVariables que no aportan información:")
print(eliminar)

columnas_eliminar = eliminar

df_train = df_train.drop(columns=columnas_eliminar)
df_val = df_val.drop(columns=columnas_eliminar)
df_test = df_test.drop(columns=columnas_eliminar)
print(f"\nColumnas que quedan después de la eliminación de las no relevantes: {df_train.columns.tolist()}")